<h1 align="center"> Airline Delays </h1>
<h2 align="center"> W261 - Final Project </h2>
<h5 align="center"> by Adam Sohn, Chandra Shekar Bikkanur, Jayesh Parikh, Tucker Anderson</h5>

<h2 align="center"> Introduction:</h2>

In this project, we have gathered airline data across United States and corresponding weather data to predict the arrival delay of a flight in minutes.

<h2 align="center"> Libraries:</h2>

We will first load the required Python, PySpark and other required libraries to run the analysis.

In [4]:
import re
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import time
import functools
import dateutil.parser
import datetime
from math import atan2, cos, sin, radians, degrees

from pyspark.sql import functions as f
from pyspark.sql.types import StructType, StructField, StringType, DoubleType, IntegerType, NullType, ShortType, DateType, BooleanType, BinaryType, TimestampType
from pyspark.sql import SQLContext
from pyspark.sql.functions import col, concat, lit, udf
from pyspark.sql import DataFrameNaFunctions
sqlContext = SQLContext(sc)

from pyspark.ml.feature import OneHotEncoderEstimator, StringIndexer, VectorAssembler
from pyspark.ml import Pipeline
from pyspark.ml.regression import LinearRegression
from pyspark.ml.regression import DecisionTreeRegressor
from pyspark.ml.regression import RandomForestRegressor
from pyspark.ml.regression import GBTRegressor
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.tuning import ParamGridBuilder, TrainValidationSplit, CrossValidator, CrossValidatorModel
from pyspark.ml.stat import ChiSquareTest

<h2 align="center">Data:</h2>

For this analysis, we are going to import airlines data, weather data and airport codes data across United States.

In [6]:
#Read in airlines, weather, stations, airport codes dataset
airlines = spark.read.option("header", "true").parquet(f"dbfs:/mnt/mids-w261/data/datasets_final_project/parquet_airlines_data/201*.parquet")
weather_parquet = spark.read.option("header", "true")\
                      .parquet(f"dbfs:/mnt/mids-w261/data/datasets_final_project/new_weather_parquet_177/weather201*a.parquet")
stations = spark.read.option("header", "true").csv("dbfs:/mnt/mids-w261/data/DEMO8/gsod/stations.csv.gz")
airport_codes = spark.read.csv('/FileStore/tables/airport_codes.csv', header="true", inferSchema="true")
airport_codes = airport_codes.selectExpr("`IATA Code` as code", "Latitude as lat", "Longitude as lon")

<h2 align="center">Airlines & Weather Data Merge:</h2>

We will merge airlines and weather data sets to form a composite dataset.

In [8]:
def is_Weekend(x):
  """
  Function to determine if a given day of the week is a weekend_day(Friday, Saturday, Sunday)
  """
  if   x < 5: 
    return 0
  else: 
    return 1

def is_RushHour(x):
  """
  Function to determine if a given time of the day is rush hour (1600-2100)
  """
  if (x != None) and (x >= 1600) and (x <= 2100): 
    return 1
  else: 
    return 0
 
def preprocessAirlines(df):
  cols_to_keep = ['MONTH', 'DAY_OF_WEEK', 'FL_DATE', 'OP_UNIQUE_CARRIER', 'ORIGIN', 'DEST', 'DEP_DELAY', 'DEP_TIME_BLK', 'ARR_DELAY', 'ARR_TIME_BLK', 'CRS_ELAPSED_TIME', 'DISTANCE',  'CARRIER_DELAY', 'WEATHER_DELAY', 'NAS_DELAY', 'SECURITY_DELAY', 'LATE_AIRCRAFT_DELAY', 'IS_WEEKEND', 'DEP_RUSH_HOUR', 'ARR_RUSH_HOUR', 'DEP_TIME', 'CRS_DEP_TIME', 'ARR_TIME', 'CRS_ARR_TIME']
  cols_to_remove = [x for x in df.columns if x not in cols_to_keep]
  df = df.orderBy("FL_DATE") 
  df = df.filter(df.CANCELLED == False)
  df = df.filter(df.DIVERTED == False)
  df = df.withColumn('CARRIER_DELAY', f.when(df.CARRIER_DELAY.isNotNull(), 1).otherwise(0))
  df = df.withColumn('WEATHER_DELAY', f.when(df.WEATHER_DELAY.isNotNull(), 1).otherwise(0))
  df = df.withColumn('NAS_DELAY', f.when(df.NAS_DELAY.isNotNull(), 1).otherwise(0))
  df = df.withColumn('SECURITY_DELAY', f.when(df.SECURITY_DELAY.isNotNull(), 1).otherwise(0))
  df = df.withColumn('LATE_AIRCRAFT_DELAY', f.when(df.LATE_AIRCRAFT_DELAY.isNotNull(), 1).otherwise(0))
  df = df.withColumn("IS_WEEKEND", f.udf(is_Weekend, IntegerType())("DAY_OF_WEEK"))
  df = df.withColumn("DEP_RUSH_HOUR", f.udf(is_RushHour, IntegerType())("DEP_TIME"))
  df = df.withColumn("ARR_RUSH_HOUR", f.udf(is_RushHour, IntegerType())("CRS_ARR_TIME"))
  df = df.fillna(0, subset=['ARR_DELAY', 'DEP_DELAY'])  
  df = df.withColumn('ORIGIN_CARRIER', concat(col("ORIGIN"), lit("_"), col("OP_UNIQUE_CARRIER")))
  df = df.withColumn('DEST_CARRIER', concat(col("DEST"), lit("_"), col("OP_UNIQUE_CARRIER")))
  preprocessAirlines_df = df.drop(*cols_to_remove)
  return preprocessAirlines_df

In [9]:
def unionAll_fn(dfs):
    return functools.reduce(lambda df1,df2: df1.union(df2.select(df1.columns)), dfs) 

def US_fn(df):
    """
    Reduce df to US only to reduce size of dataset
    """
    # US is lat/long ranges according to format: [[(lat_low, lat_high),(long_low, long_high)], [(lat_low, lat_high),(long_low, long_high)]]
    US = [[(24,49),(-125,-67)],[(17,19),(-68,-65.5)], [(13,14),(144,145)], [(15,16),(145,146)], [(-15,-14), (-171,-170)], [(18,19),(-65.4,-64)], [(18,23),(-160,-154)], [(50,175),(-170,-103)]]  

    list_df = [] #empty list for parquet parts
    parquet_part = spark.range(0).drop("id") #empty spark df

    #Filtering for individual areas in US
    for item in US:
      parquet_part = df.filter((f.col('Latitude') > item[0][0]) & (f.col('Latitude') < item[0][1]) & (f.col('Longitude') > item[1][0]) & (f.col('Longitude') < item[1][1]))
      list_df.append(parquet_part)
    
    #Appending each individual US area
    parquet_us = unionAll_fn(list_df)

    return parquet_us

def reduce_split_cols_fn(weather_parquet_us):
    """
    Reduce weather dataset to columns of interest and return split columns with comma-separated values into multiple columns for each comma-separated value.
    """
    #Reduce weather dataset to columns of interest (high level) and return split columns with comma-separated values into multiple columns for each comma-separated value.
    weather_pre_split = weather_parquet_us.select('STATION','DATE','SOURCE','LATITUDE','LONGITUDE',f.split('WND', ',').alias('WND'),f.split('VIS', ',').alias('VIS'),f.split('SLP', ',').alias('SLP'),f.split('AA1', ',').alias('AA1'))
    df_sizes_WND = weather_pre_split.select(f.size('WND').alias('WND'))
    df_sizes_VIS = weather_pre_split.select(f.size('VIS').alias('VIS'))
    df_sizes_SLP = weather_pre_split.select(f.size('SLP').alias('SLP'))
    df_sizes_AA1 = weather_pre_split.select(f.size('AA1').alias('AA1'))
    df_max_WND = df_sizes_WND.agg(f.max('WND'))
    df_max_VIS = df_sizes_VIS.agg(f.max('VIS'))
    df_max_SLP = df_sizes_SLP.agg(f.max('SLP'))
    df_max_AA1 = df_sizes_AA1.agg(f.max('AA1'))
    nb_columns_WND = df_max_WND.collect()[0][0]
    nb_columns_VIS = df_max_VIS.collect()[0][0]
    nb_columns_SLP = df_max_SLP.collect()[0][0]
    nb_columns_AA1 = df_max_AA1.collect()[0][0]
    weather_post_split = weather_pre_split.select('STATION','DATE','SOURCE','LATITUDE','LONGITUDE',*[weather_pre_split['WND'][i] for i in range(nb_columns_WND)],*[weather_pre_split['VIS'][i] for i in range(nb_columns_VIS)],*[weather_pre_split['SLP'][i] for i in range(nb_columns_SLP)],*[weather_pre_split['AA1'][i] for i in range(nb_columns_AA1)])
  
    #Filtering out data with quality issues. All string values are indicative of quality issue
    fltr_msk = [
    f.col('WND[0]') != '999',
    f.col('WND[1]') != '2',
    f.col('WND[1]') != '3',
    f.col('WND[1]') != '6',
    f.col('WND[1]') != '7',
    f.col('WND[2]') != '9',
    f.col('WND[3]') != '9999',  
    f.col('WND[4]') != '2',
    f.col('WND[4]') != '3',
    f.col('WND[4]') != '6',
    f.col('WND[4]') != '7',
    f.col('VIS[0]') != '999999',
    f.col('VIS[1]') != '2',
    f.col('VIS[1]') != '3',
    f.col('VIS[1]') != '6',
    f.col('VIS[1]') != '7',
    f.col('VIS[2]') != '9',
    f.col('VIS[3]') != '2',
    f.col('VIS[3]') != '3',
    f.col('VIS[3]') != '6',
    f.col('VIS[3]') != '7',
    f.col('SLP[0]') != '99999',
    f.col('SLP[1]') != '2',
    f.col('SLP[1]') != '3',
    f.col('SLP[1]') != '6',
    f.col('SLP[1]') != '7',
    f.col('SLP[1]') != '9',
    f.col('AA1[0]') != '99',
    f.col('AA1[1]') != '9999',
    f.col('AA1[2]') != '9',
    f.col('AA1[3]') != '2',
    f.col('AA1[3]') != '3',
    f.col('AA1[3]') != '6',
    f.col('AA1[3]') != '7'
    ]
    weather_fltr = weather_post_split
    for i in fltr_msk:
      weather_fltr = weather_fltr.filter(i)

    #Reduce weather dataset to columns of interest (low level)
    weather_fltr_drop = weather_fltr.select('STATION','DATE','SOURCE','LATITUDE','LONGITUDE','WND[0]', 'WND[3]','VIS[0]','SLP[0]','AA1[0]')
    weather_fltr_drop = weather_fltr_drop.withColumnRenamed("DATE", "TIMESTAMP")

    return weather_fltr_drop

def distinct_station_fn(weather_fltr_drop):
    """
    For df input, return distinct stations for calculating closest stations to airports
    """
    weather_fltr_drop_distinct = weather_fltr_drop.select("STATION", "LATITUDE", "LONGITUDE").distinct()
    return weather_fltr_drop_distinct

def haversine_join_station_aircode_fn(airport_codes_df, weather_df):
    """
    For df input, return haversine distance
    """
    airport_codes_df.createOrReplaceTempView('airport_codes_us')
    weather_df.createOrReplaceTempView('stations_all')
    distance_query = "(SELECT airport_codes_us.code, stations_all.STATION, airport_codes_us.lat AS airport_lat, airport_codes_us.lon AS airport_lon, ( 3959 * acos(cos(radians(airport_codes_us.lat) ) * cos( radians( stations_all.LATITUDE ) ) * cos( radians( stations_all.LONGITUDE ) - radians(airport_codes_us.lon) ) + sin(radians(airport_codes_us.lat) ) * sin( radians( stations_all.LATITUDE ) ) ) ) AS airport_station_distance FROM airport_codes_us CROSS JOIN stations_all)"
    airports_stations_distance_all = spark.sql(distance_query)
    return airports_stations_distance_all
  
def airports_closest_stations_fn(airports_stations_distance_all):
    """
    For df input, return df with closest weather stations to airports
    """

    airports_stations_distance_all.createOrReplaceTempView('airports_stations_distance')
    closest_query = "(SELECT code AS airport_code, STATION AS station_name, airport_lat, airport_lon, airport_station_distance FROM airports_stations_distance ORDER BY airport_station_distance)"
    airports_closest_stations = spark.sql(closest_query)
  
    min_distance_query = "(SELECT code AS airport_code, STATION AS station_code, airport_lat, airport_lon, airport_station_distance FROM (SELECT *, row_number() over (partition by code order by airport_station_distance ASC) as seqnum from airports_stations_distance) airports_stations_distance where seqnum = 1)"
    airports_closest_station = spark.sql(min_distance_query)    

    MAX_ALLOWABLE_WEATHER_DISTANCE = 50.0
    airports_closest_station_filtered = airports_closest_station.filter(airports_closest_station.airport_station_distance < MAX_ALLOWABLE_WEATHER_DISTANCE)
    return airports_closest_station_filtered
  
def bearingClass_fn(flight_bearing, denominations=8):
    denom = 360/denominations
        
    if (int(flight_bearing) < 0 + denom/2) or (int(flight_bearing) > (7*denom) + (denom/2)):
      flight_bearing_class = "N"
    elif int(flight_bearing) <= denom + (denom/2):
      flight_bearing_class = "NW"
    elif int(flight_bearing) <= (2*denom) + (denom/2):
      flight_bearing_class = "W"
    elif int(flight_bearing) <= (3*denom) + (denom/2):
      flight_bearing_class = "SW"
    elif int(flight_bearing) <= (4*denom) + (denom/2):
      flight_bearing_class = "S"
    elif int(flight_bearing) <= (5*denom) + (denom/2):
      flight_bearing_class = "SE"
    elif int(flight_bearing) <= (6*denom) + (denom/2):
      flight_bearing_class = "E"
    elif int(flight_bearing) <= (7*denom) + (denom/2):
      flight_bearing_class = "NE"
    else:
      flight_bearing_class = "UNK"
      
    return flight_bearing_class
  
udfBearingClass_fn = udf(bearingClass_fn, StringType())

def bearingCalculation_fn(lat_a, lon_a, lat_b, lon_b):  
    lat_a_r, lat_b_r, lon_a_r, lon_b_r = radians(lat_a), radians(lat_b), radians(lon_a), radians(lon_b)
    delta_lon = lon_b - lon_a
    delta_lon_r = lon_b_r - lon_a_r
    X = cos(lat_b_r) * sin(delta_lon_r)
    Y = cos(lat_a_r) * sin(lat_b_r) - sin(lat_a_r) * cos(lat_b_r) * cos(delta_lon_r)
  
    flight_bearing = degrees(atan2(X, Y))
        
    flight_bearing_class = bearingClass_fn(flight_bearing)
  
    return flight_bearing_class
udfBearingCalculation_fn = udf(bearingCalculation_fn, StringType())

def join_closest_weather_airlines_fn(airlines_df, airports_closest_station_filtered):

    # add closest weather station to airlines dataset
    airlines_station_origin_filtered = airlines_df.join(airports_closest_station_filtered, airlines_df.ORIGIN == airports_closest_station_filtered.airport_code, how="inner")
    airlines_station_origin_filtered = airlines_station_origin_filtered.withColumnRenamed("station_code", "ORIGIN_STATION")
    airlines_station_origin_filtered = airlines_station_origin_filtered.withColumnRenamed("airport_station_distance", "ORIGIN_STATION_DISTANCE")
    airlines_station_origin_filtered = airlines_station_origin_filtered.withColumnRenamed("airport_lat", "ORIGIN_LAT")
    airlines_station_origin_filtered = airlines_station_origin_filtered.withColumnRenamed("airport_lon", "ORIGIN_LON")
    airlines_station_origin_filtered = airlines_station_origin_filtered.drop("airport_code")

    # add closest weather station to airlines dataset
    airlines_station_filtered = airlines_station_origin_filtered.join(airports_closest_station_filtered, airlines_station_origin_filtered.DEST == airports_closest_station_filtered.airport_code, how="inner")
    airlines_station_filtered = airlines_station_filtered.withColumnRenamed("station_code", "DEST_STATION")
    airlines_station_filtered = airlines_station_filtered.withColumnRenamed("airport_station_distance", "DEST_STATION_DISTANCE")
    airlines_station_filtered = airlines_station_filtered.withColumnRenamed("airport_lat", "DEST_LAT")
    airlines_station_filtered = airlines_station_filtered.withColumnRenamed("airport_lon", "DEST_LON")
    airlines_station_filtered = airlines_station_filtered.drop("airport_code")

    #add flight bearing angle in degrees from true north (consistent with wind direction)
    airlines_station_filtered = airlines_station_filtered.withColumn("FLIGHT_BEARING", udfBearingCalculation_fn("ORIGIN_LAT","ORIGIN_LON","DEST_LAT","DEST_LON"))
    return airlines_station_filtered

def flightDateTimeCalculation_fn(flight_date, flight_time):  
    timestamp_date = str(flight_date)
    timestamp_hour = str(flight_time).zfill(4)[:-2]
    timestamp_minute = str(flight_time).zfill(4)[-2:]
  
    timestamp = timestamp_date + 'T' + timestamp_hour + ':' + timestamp_minute# + ".000+0000"
    try:
      datetime_timestamp = dateutil.parser.isoparse(timestamp)
    except ValueError:
      timestamp = timestamp_date + 'T' + '00' + ':' + timestamp_minute# + ".000+0000"
      datetime_timestamp = dateutil.parser.isoparse(timestamp)
    
    return datetime_timestamp
  
def flightDateTimeCalculationArr_fn(flight_date, flight_time_dep, flight_time_arr):  
    timestamp_dep_date = str(flight_date)
    timestamp_arr_date = str(flight_date)
  
    
    timestamp_dep_hour = str(flight_time_dep).zfill(4)[:-2]
    timestamp_dep_minute = str(flight_time_dep).zfill(4)[-2:]
    timestamp_arr_hour = str(flight_time_arr).zfill(4)[:-2]
    timestamp_arr_minute = str(flight_time_arr).zfill(4)[-2:]
    
    timestamp_dep = timestamp_dep_hour + ':' + timestamp_dep_minute
    timestamp_arr = timestamp_arr_hour + ':' + timestamp_arr_minute
    
    timestamp_dep = timestamp_dep_date + 'T' + timestamp_dep_hour + ':' + timestamp_dep_minute# + ".000+0000"
    try:
      datetime_timestamp_dep = dateutil.parser.isoparse(timestamp_dep)
    except ValueError:
      timestamp_dep = timestamp_dep_date + 'T' + '00' + ':' + timestamp_dep_minute# + ".000+0000"
      datetime_timestamp_dep = dateutil.parser.isoparse(timestamp_dep)
    
    timestamp_arr = timestamp_arr_date + 'T' + timestamp_arr_hour + ':' + timestamp_arr_minute# + ".000+0000"
    try:
      datetime_timestamp_arr = dateutil.parser.isoparse(timestamp_arr)
    except ValueError:
      timestamp_arr = timestamp_arr_date + 'T' + '00' + ':' + timestamp_arr_minute# + ".000+0000"
      datetime_timestamp_arr = dateutil.parser.isoparse(timestamp_arr)
  
    # if flight arrived a later than when started, only works if flight was less than 24 hours long:
    if datetime_timestamp_dep > datetime_timestamp_arr:
      datetime_timestamp_arr = datetime_timestamp_arr + datetime.timedelta(days=1)

    return datetime_timestamp_arr

udfFlightDateTimeCalculation_fn = udf(flightDateTimeCalculation_fn, TimestampType())
udfFlightDateTimeCalculationArr_fn = udf(flightDateTimeCalculationArr_fn, TimestampType())

def airlines_station_datetime_fn(airlines_station_filtered):
    airlines_station_datetime = airlines_station_filtered.withColumn("CRS_DEP_TIMESTAMP", udfFlightDateTimeCalculation_fn("FL_DATE","CRS_DEP_TIME"))
    airlines_station_datetime = airlines_station_datetime.withColumn("CRS_ARR_TIMESTAMP", udfFlightDateTimeCalculationArr_fn("FL_DATE","CRS_DEP_TIME", "CRS_ARR_TIME"))
    return airlines_station_datetime

def airlines_station_datetime_unix_fn(airlines_station_datetime):
    airlines_station_datetime_unix = airlines_station_datetime.withColumn("CRS_DEP_TIMESTAMP_UNIX", f.unix_timestamp("CRS_DEP_TIMESTAMP"))
    airlines_station_datetime_unix = airlines_station_datetime_unix.withColumn("CRS_ARR_TIMESTAMP_UNIX", f.unix_timestamp("CRS_ARR_TIMESTAMP"))
    airlines_station_datetime_unix = airlines_station_datetime_unix.withColumn("DEP_HOUR", f.hour("CRS_DEP_TIMESTAMP"))
    airlines_station_datetime_unix = airlines_station_datetime_unix.withColumn("ARR_HOUR", f.hour("CRS_ARR_TIMESTAMP"))
    
    return airlines_station_datetime_unix
  
def weather_fltr_datetime_fn(weather_fltr_drop):
    weather_fltr_datetime = weather_fltr_drop.withColumn("DATE_TIMESTAMP_UNIX", f.unix_timestamp("TIMESTAMP"))
    weather_fltr_datetime = weather_fltr_datetime.withColumn('DATE', f.col("TIMESTAMP").cast(DateType()))
    weather_fltr_datetime = weather_fltr_datetime.withColumn("HOUR", f.hour("TIMESTAMP"))
    
    return weather_fltr_datetime

def weather_avg_fn(weather_fltr_datetime):
    weather_fltr_datetime.createOrReplaceTempView('weather_time')
    weather_avg_query = "(SELECT STATION, DATE, HOUR, ROUND(AVG(`WND[0]`),0) AS `WND[0]`, ROUND(AVG(`WND[3]`),0) AS `WND[3]`, ROUND(AVG(`VIS[0]`),0) AS `VIS[0]`, ROUND(AVG(`SLP[0]`),0) AS `SLP[0]`, ROUND(AVG(`AA1[0]`),0) AS `AA1[0]` FROM weather_time GROUP BY STATION, DATE, HOUR)"

    weather_avg = spark.sql(weather_avg_query)
    
    weather_avg = weather_avg.withColumn("WND_CLASS[0]", udfBearingClass_fn("WND[0]"))
    weather_avg = weather_avg.drop("WND[0]")
    
    return weather_avg
  
def weather_add_values_fn(weather_avg):
    weather_fltr_datetime_origin = weather_avg.withColumnRenamed("STATION", "ORIGIN_STATION_WEATHER")
    weather_fltr_datetime_origin = weather_fltr_datetime_origin.withColumnRenamed("DATE", "ORIGIN_STATION_DATE")
    weather_fltr_datetime_origin = weather_fltr_datetime_origin.withColumnRenamed("HOUR", "ORIGIN_STATION_HOUR")
    weather_fltr_datetime_origin = weather_fltr_datetime_origin.withColumnRenamed("WND_CLASS[0]", "ORIGIN_STATION_WND[0]")
    weather_fltr_datetime_origin = weather_fltr_datetime_origin.withColumnRenamed("WND[3]", "ORIGIN_STATION_WND[3]")
    weather_fltr_datetime_origin = weather_fltr_datetime_origin.withColumnRenamed("VIS[0]", "ORIGIN_STATION_VIS[0]")
    weather_fltr_datetime_origin = weather_fltr_datetime_origin.withColumnRenamed("SLP[0]", "ORIGIN_STATION_SLP[0]")
    weather_fltr_datetime_origin = weather_fltr_datetime_origin.withColumnRenamed("AA1[0]", "ORIGIN_STATION_AA1[0]")
    weather_fltr_datetime_dest = weather_avg.withColumnRenamed("STATION", "DEST_STATION_WEATHER")
    weather_fltr_datetime_dest = weather_fltr_datetime_dest.withColumnRenamed("DATE", "DEST_STATION_DATE")
    weather_fltr_datetime_dest = weather_fltr_datetime_dest.withColumnRenamed("HOUR", "DEST_STATION_HOUR")
    weather_fltr_datetime_dest = weather_fltr_datetime_dest.withColumnRenamed("WND_CLASS[0]", "DEST_STATION_WND[0]")
    weather_fltr_datetime_dest = weather_fltr_datetime_dest.withColumnRenamed("WND[3]", "DEST_STATION_WND[3]")
    weather_fltr_datetime_dest = weather_fltr_datetime_dest.withColumnRenamed("VIS[0]", "DEST_STATION_VIS[0]")
    weather_fltr_datetime_dest = weather_fltr_datetime_dest.withColumnRenamed("SLP[0]", "DEST_STATION_SLP[0]")
    weather_fltr_datetime_dest = weather_fltr_datetime_dest.withColumnRenamed("AA1[0]", "DEST_STATION_AA1[0]")
    return weather_fltr_datetime_origin, weather_fltr_datetime_dest
  
def departure_final_fn(airlines_station_datetime_unix):
    airlines_station_datetime_unix.createOrReplaceTempView("airports_weather")
    weather_fltr_datetime_origin.createOrReplaceTempView("origin_weather")
    origin_join_query = "(SELECT * FROM airports_weather a INNER JOIN origin_weather w ON a.ORIGIN_STATION = w.ORIGIN_STATION_WEATHER AND a.FL_DATE = w.ORIGIN_STATION_DATE AND a.DEP_HOUR = w.ORIGIN_STATION_HOUR)"

    departure_final = spark.sql(origin_join_query)
    return departure_final

def airlines_weather_final_trim_fn(departure_final):
    departure_final.createOrReplaceTempView("airports_weather_dest")
    weather_fltr_datetime_dest.createOrReplaceTempView("dest_weather")
    # chnaged to join on weather @ destination airport @ departure time
    dest_join_query = "(SELECT * FROM airports_weather_dest a INNER JOIN dest_weather w ON a.DEST_STATION = w.DEST_STATION_WEATHER AND a.FL_DATE = w.DEST_STATION_DATE AND a.DEP_HOUR = w.DEST_STATION_HOUR)"

    airlines_weather_final = spark.sql(dest_join_query)
    drop_cols = ['DEST_STATION_DATE', 'DEST_STATION_HOUR', 'ORIGIN_STATION_HOUR', 'ORIGIN_STATION_DATE', 'ORIGIN_LAT', 'ORIGIN_LON', 'DEST_LAT', 'DEST_LON', 'CRS_DEP_TIMESTAMP_UNIX', 'CRS_ARR_TIMESTAMP_UNIX', 'DEP_HOUR', 'ARR_HOUR', 'ORIGIN_STATION', 'DEST_STATION', 'ORIGIN_STATION_WEATHER', 'DEST_STATION_WEATHER']
    airlines_weather_final_trim = airlines_weather_final.drop(*drop_cols)
    return airlines_weather_final_trim
  
def airlines_weather_to_parquet_fn(airlines_weather_final_trim):
    dbutils.fs.rm("dbfs:/tmp/parquet/airlines_weather_final_4_7.parquet")
    airlines_weather_final_trim.write.parquet("dbfs:/tmp/parquet/airlines_weather_final_4_7.parquet")
    return None

In [10]:
airlines_df =  preprocessAirlines(airlines)
weather_parquet_us = US_fn(weather_parquet)
weather_fltr_drop = reduce_split_cols_fn(weather_parquet_us)
weather_fltr_drop_distinct = distinct_station_fn(weather_fltr_drop)
airport_codes_us = US_fn(airport_codes)
airports_stations_distance_all = haversine_join_station_aircode_fn(airport_codes_us, weather_fltr_drop_distinct)
airports_closest_station_filtered = airports_closest_stations_fn(airports_stations_distance_all)
airports_closest_station_filtered = join_closest_weather_airlines_fn(airlines_df, airports_closest_station_filtered)
airlines_station_datetime = airlines_station_datetime_fn(airports_closest_station_filtered)
airlines_station_datetime_unix = airlines_station_datetime_unix_fn(airlines_station_datetime)  
weather_fltr_datetime = weather_fltr_datetime_fn(weather_fltr_drop)
weather_avg = weather_avg_fn(weather_fltr_datetime)
weather_fltr_datetime_origin, weather_fltr_datetime_dest = weather_add_values_fn(weather_avg)
departure_final = departure_final_fn(airlines_station_datetime_unix)
airlines_weather_final_trim = airlines_weather_final_trim_fn(departure_final)

In [11]:
#airlines_weather_final_trim.write.parquet("/FileStore/tables/airlines_weather_final_trim.parquet")
airlines_weather_final_trim = spark.read.parquet("/FileStore/tables/airlines_weather_final_trim.parquet")

In [12]:
airlines_weather_final_trim.printSchema()

root
-- MONTH: integer (nullable = true)
-- DAY_OF_WEEK: integer (nullable = true)
-- FL_DATE: date (nullable = true)
-- OP_UNIQUE_CARRIER: string (nullable = true)
-- ORIGIN: string (nullable = true)
-- DEST: string (nullable = true)
-- CRS_DEP_TIME: integer (nullable = true)
-- DEP_TIME: integer (nullable = true)
-- DEP_DELAY: double (nullable = true)
-- DEP_TIME_BLK: string (nullable = true)
-- CRS_ARR_TIME: integer (nullable = true)
-- ARR_TIME: integer (nullable = true)
-- ARR_DELAY: double (nullable = true)
-- ARR_TIME_BLK: string (nullable = true)
-- CRS_ELAPSED_TIME: double (nullable = true)
-- DISTANCE: double (nullable = true)
-- CARRIER_DELAY: integer (nullable = true)
-- WEATHER_DELAY: integer (nullable = true)
-- NAS_DELAY: integer (nullable = true)
-- SECURITY_DELAY: integer (nullable = true)
-- LATE_AIRCRAFT_DELAY: integer (nullable = true)
-- IS_WEEKEND: integer (nullable = true)
-- DEP_RUSH_HOUR: integer (nullable = true)
-- ARR_RUSH_HOUR: integer (nullable = true)
-- ORIGIN_CARRIER: string (nullable = true)
-- DEST_CARRIER: string (nullable = true)
-- ORIGIN_STATION_DISTANCE: double (nullable = true)
-- DEST_STATION_DISTANCE: double (nullable = true)
-- FLIGHT_BEARING: string (nullable = true)
-- CRS_DEP_TIMESTAMP: timestamp (nullable = true)
-- CRS_ARR_TIMESTAMP: timestamp (nullable = true)
-- ORIGIN_STATION_WND[3]: double (nullable = true)
-- ORIGIN_STATION_VIS[0]: double (nullable = true)
-- ORIGIN_STATION_SLP[0]: double (nullable = true)
-- ORIGIN_STATION_AA1[0]: double (nullable = true)
-- ORIGIN_STATION_WND[0]: string (nullable = true)
-- DEST_STATION_WND[3]: double (nullable = true)
-- DEST_STATION_VIS[0]: double (nullable = true)
-- DEST_STATION_SLP[0]: double (nullable = true)
-- DEST_STATION_AA1[0]: double (nullable = true)
-- DEST_STATION_WND[0]: string (nullable = true)

## Data Split:

In [14]:
airlines_train, airlines_val, airlines_test = airlines_weather_final_trim.randomSplit([0.8,0.1,0.1], seed = 2020)

In [15]:
train_cnt = airlines_train.count()
val_cnt = airlines_val.count()
test_cnt = airlines_test.count()
total_cnt = train_cnt + val_cnt + test_cnt
print('airlines_train records: {}\n airlines_val records: {}\n  airlines_test records: {}\n total records: {}'.format(train_cnt, val_cnt, test_cnt, total_cnt) )

airlines_train records: 109431
 airlines_val records: 13766
 airlines_test records: 13630
 total records: 136827

## Feature Engineering & Feature Selection:

In [17]:
def featureSelection(df):
  cols_to_keep = ['MONTH', 'DAY_OF_WEEK', 'DEP_DELAY', 'DEP_TIME_BLK', 'ARR_DELAY', 'ARR_TIME_BLK', 'CRS_ELAPSED_TIME', 'DISTANCE',  'CARRIER_DELAY', 'WEATHER_DELAY', 'NAS_DELAY', 'SECURITY_DELAY', 'LATE_AIRCRAFT_DELAY', 'IS_WEEKEND', 'DEP_RUSH_HOUR', 'ARR_RUSH_HOUR','FLIGHT_BEARING', 'ORIGIN_CARRIER', 'DEST_CARRIER', 'ORIGIN_STATION_WND_DIR', 'ORIGIN_STATION_VIS', 'ORIGIN_STATION_SLP','ORIGIN_STATION_AA1',    'ORIGIN_STATION_WND', 'DEST_STATION_WND_DIR', 'DEST_STATION_VIS', 'DEST_STATION_SLP', 'DEST_STATION_AA1',  'DEST_STATION_WND']
  cols_to_remove = [x for x in df.columns if x not in cols_to_keep]
  df = df.withColumnRenamed("ORIGIN_STATION_WND[0]", "ORIGIN_STATION_WND_DIR")
  df = df.withColumnRenamed("ORIGIN_STATION_VIS[0]", "ORIGIN_STATION_VIS")
  df = df.withColumnRenamed("ORIGIN_STATION_SLP[0]", "ORIGIN_STATION_SLP")
  df = df.withColumnRenamed("ORIGIN_STATION_AA1[0]", "ORIGIN_STATION_AA1")
  df = df.withColumnRenamed("ORIGIN_STATION_WND[3]", "ORIGIN_STATION_WND")
  
  df = df.withColumnRenamed("DEST_STATION_WND[0]", "DEST_STATION_WND_DIR")
  df = df.withColumnRenamed("DEST_STATION_VIS[0]", "DEST_STATION_VIS")
  df = df.withColumnRenamed("DEST_STATION_SLP[0]", "DEST_STATION_SLP")
  df = df.withColumnRenamed("DEST_STATION_AA1[0]", "DEST_STATION_AA1")
  df = df.withColumnRenamed("DEST_STATION_WND[3]", "DEST_STATION_WND")  
  
  featureSelection_df = df.drop(*cols_to_remove)
  return featureSelection_df

In [18]:
airlines_train_df =  featureSelection(airlines_train)

In [19]:
numeric_features = [x[0] for x in airlines_train_df.dtypes if x[1] == 'int' or x[1] == 'double']
numeric_features.remove('ARR_DELAY')
numeric_features

Out[17]: ['MONTH',
 'DAY_OF_WEEK',
 'DEP_DELAY',
 'CRS_ELAPSED_TIME',
 'DISTANCE',
 'CARRIER_DELAY',
 'WEATHER_DELAY',
 'NAS_DELAY',
 'SECURITY_DELAY',
 'LATE_AIRCRAFT_DELAY',
 'IS_WEEKEND',
 'DEP_RUSH_HOUR',
 'ARR_RUSH_HOUR',
 'ORIGIN_STATION_WND',
 'ORIGIN_STATION_VIS',
 'ORIGIN_STATION_SLP',
 'ORIGIN_STATION_AA1',
 'DEST_STATION_WND',
 'DEST_STATION_VIS',
 'DEST_STATION_SLP',
 'DEST_STATION_AA1']

In [20]:
categorical_features = [x[0] for x in airlines_train_df.dtypes if x[1] == 'string']
categorical_features

Out[18]: ['DEP_TIME_BLK',
 'ARR_TIME_BLK',
 'ORIGIN_CARRIER',
 'DEST_CARRIER',
 'FLIGHT_BEARING',
 'ORIGIN_STATION_WND_DIR',
 'DEST_STATION_WND_DIR']

In [21]:
stages = []
for categoricalCol in categorical_features:
    stringIndexer = StringIndexer(inputCol = categoricalCol, outputCol = categoricalCol + 'Index', handleInvalid="keep")
    encoder = OneHotEncoderEstimator(inputCols=[stringIndexer.getOutputCol()], outputCols=[categoricalCol + "classVec"])
    stages += [stringIndexer, encoder]
assemblerInputs = [c + "classVec" for c in categorical_features] + numeric_features
assembler = VectorAssembler(inputCols=assemblerInputs, outputCol="features", handleInvalid="keep")
stages += [assembler]

In [22]:
pipeline = Pipeline().setStages(stages).fit(airlines_train_df)
vector_airlines_train_df = pipeline.transform(airlines_train_df)
vector_airlines_train_df.printSchema()

root
-- MONTH: integer (nullable = true)
-- DAY_OF_WEEK: integer (nullable = true)
-- DEP_DELAY: double (nullable = true)
-- DEP_TIME_BLK: string (nullable = true)
-- ARR_DELAY: double (nullable = true)
-- ARR_TIME_BLK: string (nullable = true)
-- CRS_ELAPSED_TIME: double (nullable = true)
-- DISTANCE: double (nullable = true)
-- CARRIER_DELAY: integer (nullable = true)
-- WEATHER_DELAY: integer (nullable = true)
-- NAS_DELAY: integer (nullable = true)
-- SECURITY_DELAY: integer (nullable = true)
-- LATE_AIRCRAFT_DELAY: integer (nullable = true)
-- IS_WEEKEND: integer (nullable = true)
-- DEP_RUSH_HOUR: integer (nullable = true)
-- ARR_RUSH_HOUR: integer (nullable = true)
-- ORIGIN_CARRIER: string (nullable = true)
-- DEST_CARRIER: string (nullable = true)
-- FLIGHT_BEARING: string (nullable = true)
-- ORIGIN_STATION_WND: double (nullable = true)
-- ORIGIN_STATION_VIS: double (nullable = true)
-- ORIGIN_STATION_SLP: double (nullable = true)
-- ORIGIN_STATION_AA1: double (nullable = true)
-- ORIGIN_STATION_WND_DIR: string (nullable = true)
-- DEST_STATION_WND: double (nullable = true)
-- DEST_STATION_VIS: double (nullable = true)
-- DEST_STATION_SLP: double (nullable = true)
-- DEST_STATION_AA1: double (nullable = true)
-- DEST_STATION_WND_DIR: string (nullable = true)
-- DEP_TIME_BLKIndex: double (nullable = false)
-- DEP_TIME_BLKclassVec: vector (nullable = true)
-- ARR_TIME_BLKIndex: double (nullable = false)
-- ARR_TIME_BLKclassVec: vector (nullable = true)
-- ORIGIN_CARRIERIndex: double (nullable = false)
-- ORIGIN_CARRIERclassVec: vector (nullable = true)
-- DEST_CARRIERIndex: double (nullable = false)
-- DEST_CARRIERclassVec: vector (nullable = true)
-- FLIGHT_BEARINGIndex: double (nullable = false)
-- FLIGHT_BEARINGclassVec: vector (nullable = true)
-- ORIGIN_STATION_WND_DIRIndex: double (nullable = false)
-- ORIGIN_STATION_WND_DIRclassVec: vector (nullable = true)
-- DEST_STATION_WND_DIRIndex: double (nullable = false)
-- DEST_STATION_WND_DIRclassVec: vector (nullable = true)
-- features: vector (nullable = true)

In [23]:
train_df = vector_airlines_train_df.select(col("ARR_DELAY").alias("label"), col("features"))
train_df.show(2)

+-----+--------------------+
label| features|
+-----+--------------------+
 -4.0|(3813,[17,37,433,...|
 9.0|(3813,[17,37,412,...|
+-----+--------------------+
only showing top 2 rows

In [24]:
airlines_val_df =  featureSelection(airlines_val)
vector_airlines_val_df = pipeline.transform(airlines_val_df)
val_df = vector_airlines_val_df.select(col("ARR_DELAY").alias("label"), col("features"))

## Models:
To predict `ARRIVAL_DELAY` from the dataset, we are going to consider below supervised machine learning algorithms using cross validation.

1. Linear Regression
2. Decision Tree Regressor
3. Random Forest Regressor
4. Gradient Boosted Tree Regressor

### Linear Regression:

In [27]:
lr = LinearRegression(featuresCol = 'features', labelCol='label')
paramGrid_lr = ParamGridBuilder() \
   .addGrid(lr.regParam, [0.1, 0.01, 0.001]) \
   .addGrid(lr.maxIter, [10, 20]) \
   .addGrid(lr.elasticNetParam, [0.0, 0.5, 1.0]) \
   .build() 

crossval_lr = CrossValidator(estimator=lr,
                          estimatorParamMaps=paramGrid_lr,
                          evaluator=RegressionEvaluator(),
                          numFolds=5) 

cvModel_lr = crossval_lr.fit(train_df)

MLlib will automatically track trials in MLflow. After your tuning fit() call has completed, view the MLflow UI to see logged runs.

In [28]:
regression_evaluator_r2 = RegressionEvaluator(predictionCol="prediction", labelCol="label",metricName="r2")
regression_evaluator_rmse = RegressionEvaluator(predictionCol="prediction", labelCol="label",metricName="rmse")
regression_evaluator_mae = RegressionEvaluator(predictionCol="prediction", labelCol="label",metricName="mae")

regression_metrics_list = []

In [29]:
# Save and Load CrossValidatorModel
cvModel_lr.write().overwrite().save('/FileStore/tables/cvModel_lr')

In [30]:
saved_cvModel_lr = CrossValidatorModel.load('/FileStore/tables/cvModel_lr')

In [31]:
# train_df evaluation metrics
lr_predictions_train = saved_cvModel_lr.transform(train_df)
lr_train_r2 = regression_evaluator_r2.evaluate(lr_predictions_train)
lr_train_rmse = regression_evaluator_rmse.evaluate(lr_predictions_train)
lr_train_mae = regression_evaluator_mae.evaluate(lr_predictions_train)
regression_metrics_list.append(["LinearRegression_TrainData_CV", lr_train_r2, lr_train_rmse, lr_train_mae ])

#  val_df evaluation metrics
lr_predictions_val = saved_cvModel_lr.transform(val_df)
lr_val_r2 = regression_evaluator_r2.evaluate(lr_predictions_val)
lr_val_rmse = regression_evaluator_rmse.evaluate(lr_predictions_val)
lr_val_mae = regression_evaluator_mae.evaluate(lr_predictions_val)
regression_metrics_list.append(["LinearRegression_ValData_CV", lr_val_r2, lr_val_rmse, lr_val_mae ])

In [32]:
bestLRModel = cvModel_lr.bestModel
bestParams = bestLRModel.extractParamMap()
bestParams

Out[31]: {Param(parent='LinearRegression_39888e57d903', name='aggregationDepth', doc='suggested depth for treeAggregate (>= 2)'): 2,
 Param(parent='LinearRegression_39888e57d903', name='elasticNetParam', doc='the ElasticNet mixing parameter, in range [0, 1]. For alpha = 0, the penalty is an L2 penalty. For alpha = 1, it is an L1 penalty'): 0.5,
 Param(parent='LinearRegression_39888e57d903', name='epsilon', doc='The shape parameter to control the amount of robustness. Must be > 1.0.'): 1.35,
 Param(parent='LinearRegression_39888e57d903', name='featuresCol', doc='features column name'): 'features',
 Param(parent='LinearRegression_39888e57d903', name='fitIntercept', doc='whether to fit an intercept term'): True,
 Param(parent='LinearRegression_39888e57d903', name='labelCol', doc='label column name'): 'label',
 Param(parent='LinearRegression_39888e57d903', name='loss', doc='The loss function to be optimized. Supported options: squaredError, huber. (Default squaredError)'): 'squaredError',
 Param(parent='LinearRegression_39888e57d903', name='maxIter', doc='maximum number of iterations (>= 0)'): 20,
 Param(parent='LinearRegression_39888e57d903', name='predictionCol', doc='prediction column name'): 'prediction',
 Param(parent='LinearRegression_39888e57d903', name='regParam', doc='regularization parameter (>= 0)'): 0.1,
 Param(parent='LinearRegression_39888e57d903', name='solver', doc='The solver algorithm for optimization. Supported options: auto, normal, l-bfgs. (Default auto)'): 'auto',
 Param(parent='LinearRegression_39888e57d903', name='standardization', doc='whether to standardize the training features before fitting the model'): True,
 Param(parent='LinearRegression_39888e57d903', name='tol', doc='the convergence tolerance for iterative algorithms (>= 0)'): 1e-06}

### Decision Tree Regressor

In [34]:
dt = DecisionTreeRegressor(featuresCol="features", labelCol='label') 

paramGrid_dt = ParamGridBuilder()\
    .addGrid(dt.maxBins, [16, 32]) \
    .addGrid(dt.maxDepth, [5, 10]) \
    .addGrid(dt.minInstancesPerNode, [1, 5]) \
    .build()  

crossval_dt = CrossValidator(estimator=dt,
                          estimatorParamMaps=paramGrid_dt,
                          evaluator=RegressionEvaluator(),
                          numFolds=5) 

cvModel_dt = crossval_dt.fit(train_df)

MLlib will automatically track trials in MLflow. After your tuning fit() call has completed, view the MLflow UI to see logged runs.

In [35]:
# Save and Load CrossValidatorModel
cvModel_dt.write().overwrite().save('/FileStore/tables/cvModel_dt')

In [36]:
saved_cvModel_dt = CrossValidatorModel.load('/FileStore/tables/cvModel_dt')

In [37]:
# train_df evaluation metrics
dt_predictions_train = saved_cvModel_dt.transform(train_df)
dt_train_r2 = regression_evaluator_r2.evaluate(dt_predictions_train)
dt_train_rmse = regression_evaluator_rmse.evaluate(dt_predictions_train)
dt_train_mae = regression_evaluator_mae.evaluate(dt_predictions_train)
regression_metrics_list.append(["DecisionTreeRegressor_TrainData_CV", dt_train_r2, dt_train_rmse, dt_train_mae ])

# val_df evaluation metrics
dt_predictions_val = saved_cvModel_dt.transform(val_df)
dt_val_r2 = regression_evaluator_r2.evaluate(dt_predictions_val)
dt_val_rmse = regression_evaluator_rmse.evaluate(dt_predictions_val)
dt_val_mae = regression_evaluator_mae.evaluate(dt_predictions_val)
regression_metrics_list.append(["DecisionTreeRegressor_ValData_CV", dt_val_r2, dt_val_rmse, dt_val_mae ])

In [38]:
bestDTModel = cvModel_dt.bestModel
bestParams_dt = bestDTModel.extractParamMap()
bestParams_dt

Out[36]: {Param(parent='DecisionTreeRegressor_75978e7a50e9', name='cacheNodeIds', doc='If false, the algorithm will pass trees to executors to match instances with nodes. If true, the algorithm will cache node IDs for each instance. Caching can speed up training of deeper trees.'): False,
 Param(parent='DecisionTreeRegressor_75978e7a50e9', name='checkpointInterval', doc='set checkpoint interval (>= 1) or disable checkpoint (-1). E.g. 10 means that the cache will get checkpointed every 10 iterations. Note: this setting will be ignored if the checkpoint directory is not set in the SparkContext'): 10,
 Param(parent='DecisionTreeRegressor_75978e7a50e9', name='featuresCol', doc='features column name'): 'features',
 Param(parent='DecisionTreeRegressor_75978e7a50e9', name='impurity', doc='Criterion used for information gain calculation (case-insensitive). Supported options: variance'): 'variance',
 Param(parent='DecisionTreeRegressor_75978e7a50e9', name='labelCol', doc='label column name'): 'label',
 Param(parent='DecisionTreeRegressor_75978e7a50e9', name='maxBins', doc='Max number of bins for discretizing continuous features. Must be at least 2 and at least number of categories for any categorical feature.'): 32,
 Param(parent='DecisionTreeRegressor_75978e7a50e9', name='maxDepth', doc='Maximum depth of the tree. (Nonnegative) E.g., depth 0 means 1 leaf node; depth 1 means 1 internal node + 2 leaf nodes.'): 5,
 Param(parent='DecisionTreeRegressor_75978e7a50e9', name='maxMemoryInMB', doc='Maximum memory in MB allocated to histogram aggregation.'): 256,
 Param(parent='DecisionTreeRegressor_75978e7a50e9', name='minInfoGain', doc='Minimum information gain for a split to be considered at a tree node.'): 0.0,
 Param(parent='DecisionTreeRegressor_75978e7a50e9', name='minInstancesPerNode', doc='Minimum number of instances each child must have after split. If a split causes the left or right child to have fewer than minInstancesPerNode, the split will be discarded as invalid. Must be at least 1.'): 5,
 Param(parent='DecisionTreeRegressor_75978e7a50e9', name='predictionCol', doc='prediction column name'): 'prediction',
 Param(parent='DecisionTreeRegressor_75978e7a50e9', name='seed', doc='random seed'): -1407754390808368278}

### Random Forest Regressor

In [40]:
rf = RandomForestRegressor(featuresCol="features", labelCol='label')

paramGrid_rf = ParamGridBuilder()\
    .addGrid(rf.maxBins, [16, 32]) \
    .addGrid(rf.numTrees, [20, 40]) \
    .addGrid(rf.minInstancesPerNode, [1, 5]) \
    .build()  

crossval_rf = CrossValidator(estimator=rf,
                          estimatorParamMaps=paramGrid_rf,
                          evaluator=RegressionEvaluator(),
                          numFolds=5) 

cvModel_rf = crossval_rf.fit(train_df)

MLlib will automatically track trials in MLflow. After your tuning fit() call has completed, view the MLflow UI to see logged runs.

In [41]:
  # Save and Load CrossValidatorModel
cvModel_rf.write().overwrite().save('/FileStore/tables/cvModel_rf')


In [42]:
saved_cvModel_rf = CrossValidatorModel.load('/FileStore/tables/cvModel_rf')

In [43]:
rf_predictions_train = saved_cvModel_rf.transform(train_df)
rf_train_r2 = regression_evaluator_r2.evaluate(rf_predictions_train)
rf_train_rmse = regression_evaluator_rmse.evaluate(rf_predictions_train)
rf_train_mae = regression_evaluator_mae.evaluate(rf_predictions_train)
regression_metrics_list.append(["RandomForestRegressor_TrainData_CV", rf_train_r2, rf_train_rmse, rf_train_mae ])

# val_df evaluation metrics
rf_predictions_val = saved_cvModel_rf.transform(val_df)
rf_val_r2 = regression_evaluator_r2.evaluate(rf_predictions_val)
rf_val_rmse = regression_evaluator_rmse.evaluate(rf_predictions_val)
rf_val_mae = regression_evaluator_mae.evaluate(rf_predictions_val)
regression_metrics_list.append(["RandomForestRegressor_ValData_CV", rf_val_r2, rf_val_rmse, rf_val_mae ])

In [44]:
bestRFModel = cvModel_rf.bestModel
bestParams_rf = bestRFModel.extractParamMap()
bestParams_rf

Out[41]: {Param(parent='RandomForestRegressor_b2356b0807d2', name='cacheNodeIds', doc='If false, the algorithm will pass trees to executors to match instances with nodes. If true, the algorithm will cache node IDs for each instance. Caching can speed up training of deeper trees.'): False,
 Param(parent='RandomForestRegressor_b2356b0807d2', name='checkpointInterval', doc='set checkpoint interval (>= 1) or disable checkpoint (-1). E.g. 10 means that the cache will get checkpointed every 10 iterations. Note: this setting will be ignored if the checkpoint directory is not set in the SparkContext'): 10,
 Param(parent='RandomForestRegressor_b2356b0807d2', name='featureSubsetStrategy', doc='The number of features to consider for splits at each tree node. Supported options: auto, all, onethird, sqrt, log2, (0.0-1.0], [1-n].'): 'auto',
 Param(parent='RandomForestRegressor_b2356b0807d2', name='featuresCol', doc='features column name'): 'features',
 Param(parent='RandomForestRegressor_b2356b0807d2', name='impurity', doc='Criterion used for information gain calculation (case-insensitive). Supported options: variance'): 'variance',
 Param(parent='RandomForestRegressor_b2356b0807d2', name='labelCol', doc='label column name'): 'label',
 Param(parent='RandomForestRegressor_b2356b0807d2', name='maxBins', doc='Max number of bins for discretizing continuous features. Must be at least 2 and at least number of categories for any categorical feature.'): 32,
 Param(parent='RandomForestRegressor_b2356b0807d2', name='maxDepth', doc='Maximum depth of the tree. (Nonnegative) E.g., depth 0 means 1 leaf node; depth 1 means 1 internal node + 2 leaf nodes.'): 5,
 Param(parent='RandomForestRegressor_b2356b0807d2', name='maxMemoryInMB', doc='Maximum memory in MB allocated to histogram aggregation.'): 256,
 Param(parent='RandomForestRegressor_b2356b0807d2', name='minInfoGain', doc='Minimum information gain for a split to be considered at a tree node.'): 0.0,
 Param(parent='RandomForestRegressor_b2356b0807d2', name='minInstancesPerNode', doc='Minimum number of instances each child must have after split. If a split causes the left or right child to have fewer than minInstancesPerNode, the split will be discarded as invalid. Must be at least 1.'): 5,
 Param(parent='RandomForestRegressor_b2356b0807d2', name='numTrees', doc='Number of trees to train (at least 1)'): 20,
 Param(parent='RandomForestRegressor_b2356b0807d2', name='predictionCol', doc='prediction column name'): 'prediction',
 Param(parent='RandomForestRegressor_b2356b0807d2', name='seed', doc='random seed'): 2502083311556356884,
 Param(parent='RandomForestRegressor_b2356b0807d2', name='subsamplingRate', doc='Fraction of the training data used for learning each decision tree, in range (0, 1].'): 1.0}

### Gradient-Boosted Trees

In [46]:
 gbt = GBTRegressor(featuresCol="features", labelCol='label')

paramGrid_gbt = ParamGridBuilder()\
    .addGrid(gbt.maxBins, [10, 32]) \
    .addGrid(gbt.minInstancesPerNode, [1, 5]) \
    .build()  

crossval_gbt = CrossValidator(estimator=gbt,
                          estimatorParamMaps=paramGrid_gbt,
                          evaluator=RegressionEvaluator(),
                          numFolds=5) 

cvModel_gbt = crossval_gbt.fit(train_df)

MLlib will automatically track trials in MLflow. After your tuning fit() call has completed, view the MLflow UI to see logged runs.

In [47]:
# Save and Load CrossValidatorModel
cvModel_gbt.write().overwrite().save('/FileStore/tables/cvModel_gbt')


In [48]:
saved_cvModel_gbt = CrossValidatorModel.load('/FileStore/tables/cvModel_gbt')

In [49]:
gbt_predictions_train = saved_cvModel_gbt.transform(train_df)
gbt_train_r2 = regression_evaluator_r2.evaluate(gbt_predictions_train)
gbt_train_rmse = regression_evaluator_rmse.evaluate(gbt_predictions_train)
gbt_train_mae = regression_evaluator_mae.evaluate(gbt_predictions_train)
regression_metrics_list.append(["GradientBoostedTreeRegressor_TrainData_CV", gbt_train_r2, gbt_train_rmse, gbt_train_mae ])

# val_df evaluation metrics
gbt_predictions_val = saved_cvModel_gbt.transform(val_df)
gbt_val_r2 = regression_evaluator_r2.evaluate(gbt_predictions_val)
gbt_val_rmse = regression_evaluator_rmse.evaluate(gbt_predictions_val)
gbt_val_mae = regression_evaluator_mae.evaluate(gbt_predictions_val)
regression_metrics_list.append(["GradientBoostedTreeRegressor_ValData_CV", gbt_val_r2, gbt_val_rmse, gbt_val_mae ])

In [50]:
bestGBTModel = cvModel_gbt.bestModel
bestParams_gbt = bestGBTModel.extractParamMap()
bestParams_gbt

Out[46]: {Param(parent='GBTRegressor_47615c1899f0', name='cacheNodeIds', doc='If false, the algorithm will pass trees to executors to match instances with nodes. If true, the algorithm will cache node IDs for each instance. Caching can speed up training of deeper trees.'): False,
 Param(parent='GBTRegressor_47615c1899f0', name='checkpointInterval', doc='set checkpoint interval (>= 1) or disable checkpoint (-1). E.g. 10 means that the cache will get checkpointed every 10 iterations. Note: this setting will be ignored if the checkpoint directory is not set in the SparkContext'): 10,
 Param(parent='GBTRegressor_47615c1899f0', name='featureSubsetStrategy', doc='The number of features to consider for splits at each tree node. Supported options: auto, all, onethird, sqrt, log2, (0.0-1.0], [1-n].'): 'all',
 Param(parent='GBTRegressor_47615c1899f0', name='featuresCol', doc='features column name'): 'features',
 Param(parent='GBTRegressor_47615c1899f0', name='impurity', doc='Criterion used for information gain calculation (case-insensitive). Supported options: variance'): 'variance',
 Param(parent='GBTRegressor_47615c1899f0', name='labelCol', doc='label column name'): 'label',
 Param(parent='GBTRegressor_47615c1899f0', name='lossType', doc='Loss function which GBT tries to minimize (case-insensitive). Supported options: squared, absolute'): 'squared',
 Param(parent='GBTRegressor_47615c1899f0', name='maxBins', doc='Max number of bins for discretizing continuous features. Must be at least 2 and at least number of categories for any categorical feature.'): 32,
 Param(parent='GBTRegressor_47615c1899f0', name='maxDepth', doc='Maximum depth of the tree. (Nonnegative) E.g., depth 0 means 1 leaf node; depth 1 means 1 internal node + 2 leaf nodes.'): 5,
 Param(parent='GBTRegressor_47615c1899f0', name='maxIter', doc='maximum number of iterations (>= 0)'): 20,
 Param(parent='GBTRegressor_47615c1899f0', name='maxMemoryInMB', doc='Maximum memory in MB allocated to histogram aggregation.'): 256,
 Param(parent='GBTRegressor_47615c1899f0', name='minInfoGain', doc='Minimum information gain for a split to be considered at a tree node.'): 0.0,
 Param(parent='GBTRegressor_47615c1899f0', name='minInstancesPerNode', doc='Minimum number of instances each child must have after split. If a split causes the left or right child to have fewer than minInstancesPerNode, the split will be discarded as invalid. Must be at least 1.'): 5,
 Param(parent='GBTRegressor_47615c1899f0', name='predictionCol', doc='prediction column name'): 'prediction',
 Param(parent='GBTRegressor_47615c1899f0', name='seed', doc='random seed'): -6682481135904123338,
 Param(parent='GBTRegressor_47615c1899f0', name='stepSize', doc='Step size (a.k.a. learning rate) in interval (0, 1] for shrinking the contribution of each estimator.'): 0.1,
 Param(parent='GBTRegressor_47615c1899f0', name='subsamplingRate', doc='Fraction of the training data used for learning each decision tree, in range (0, 1].'): 1.0}

## Results:

In [52]:
regression_metrics_df = pd.DataFrame(regression_metrics_list, columns = ['Model_Data' , 'R^2', 'RMSE', 'MAE']) 
display(regression_metrics_df)

Model_Data,R^2,RMSE,MAE
LinearRegression_TrainData_CV,0.9541653484453827,13.653203778494067,9.7520324715285
LinearRegression_ValData_CV,0.9511865031953709,14.076652923054516,10.060869645137496
DecisionTreeRegressor_TrainData_CV,0.7428608585734324,32.33863614551219,12.027202731711183
DecisionTreeRegressor_ValData_CV,0.7163977636111757,33.93003193051067,12.027263801228713
RandomForestRegressor_TrainData_CV,0.7040300351194754,34.80205824863998,13.619987675943355
RandomForestRegressor_ValData_CV,0.7269773797234378,30.901460466551487,13.260998046918584
GradientBoostedTreeRegressor_TrainData_CV,0.7661766262227564,30.933215728338393,11.943562458371941
GradientBoostedTreeRegressor_ValData_CV,0.7808978161029072,27.68235562503211,11.585786438285645


From above results, we can see that Linear Regression performed well on the training and validation data. We will consider Linear Regression model for our inference on test data.

## Inference:
We will now predict the `ARRIVAL_DELAY` for test data using Linear Regression model that we trained using cross validation and hyperparameter tuning.

In [55]:
airlines_test_df =  featureSelection(airlines_test)
vector_airlines_test_df = pipeline.transform(airlines_test_df)
test_df = vector_airlines_test_df.select(col("ARR_DELAY").alias("label"), col("features"))

In [56]:
# # test_df evaluation metrics
lr_predictions_test = saved_cvModel_lr.transform(test_df)
lr_test_r2 = regression_evaluator_r2.evaluate(lr_predictions_test)
lr_test_rmse = regression_evaluator_rmse.evaluate(lr_predictions_test)
lr_test_mae = regression_evaluator_mae.evaluate(lr_predictions_test)
regression_metrics_list.append(["LinearRegression_TestData_CV", lr_test_r2, lr_test_rmse, lr_test_mae ])

In [57]:
regression_metrics_test_df = pd.DataFrame(regression_metrics_list, columns = ['Model_Data' , 'R^2', 'RMSE', 'MAE']) 
display(regression_metrics_test_df)

Model_Data,R^2,RMSE,MAE
LinearRegression_TrainData_CV,0.9541653484453827,13.653203778494067,9.7520324715285
LinearRegression_ValData_CV,0.9511865031953709,14.076652923054516,10.060869645137496
DecisionTreeRegressor_TrainData_CV,0.7428608585734324,32.33863614551219,12.027202731711183
DecisionTreeRegressor_ValData_CV,0.7163977636111757,33.93003193051067,12.027263801228713
RandomForestRegressor_TrainData_CV,0.7040300351194754,34.80205824863998,13.619987675943355
RandomForestRegressor_ValData_CV,0.7269773797234378,30.901460466551487,13.260998046918584
GradientBoostedTreeRegressor_TrainData_CV,0.7661766262227564,30.933215728338393,11.943562458371941
GradientBoostedTreeRegressor_ValData_CV,0.7808978161029072,27.68235562503211,11.585786438285645
LinearRegression_TestData_CV,0.9549801363193048,13.624266345766586,9.759998821054705


## Conclusion:

In [59]:
r = ChiSquareTest.test(train_df, "features", "label").head()
print("pValues: " + str(r.pValues))


pValues: [0.0,0.9949407816654531,0.0001877435673570682,0.0,0.5368419112619083,0.9696244354022874,0.9906732953717684,0.017326564022009272,0.0,0.40158662165652115,1.4210854715202004e-14,0.0,0.0,0.9999989394047251,0.0,1.0436256491219709e-06,6.551279341238114e-08,0.0,0.0,0.9174742437094026,1.8550938563066666e-11,0.8791680251357478,0.005216495959729395,0.11913122461806236,0.999027982871304,0.0,0.0,0.6404344944457216,0.9630654319712737,0.5263403210473494,0.0,6.282103140331508e-05,0.0,1.347393308037681e-10,1.7309487176930816e-12,0.0,1.2581398445288272e-07,0.0,4.9858099123722965e-06,0.0,1.0,0.0009603465589240745,0.003689745080685758,0.9998775308237945,0.1369332759123123,0.9999890253486495,0.0,0.0,1.0,1.0,0.8504724615042023,0.9999997894784196,1.4840351170164467e-11,1.0,0.0,1.0,0.99999999879491,1.0,7.325730542184772e-07,1.0,4.451994328746878e-14,1.0,0.0,1.0,0.37631197853800313,0.0,0.9999999999999999,0.9999966773475657,0.9999999999990292,0.9999999999801963,1.0,1.0,0.0861256350725762,0.9999999999939225,0.9999999999999942,0.2281531861716375,0.9999999994781174,1.0,0.0,0.9999999999999992,0.08252198719013948,1.0,1.0,1.0,0.0,0.0,1.0,1.0,0.9671000843951564,0.9999999987204861,0.3660184052672727,1.0,1.0,1.0,1.0,7.456153472418237e-10,1.0,0.9851585487497929,0.0,1.0,0.003712159430183992,1.0,0.9986531172279666,1.0,0.002405326762999982,1.0,0.9999527684644768,0.9999999999999803,1.0,1.1102230246251565e-16,1.0,0.08014019727124355,0.9999999126809326,1.0,1.0,0.0,0.9968741339245167,0.685338618585668,1.0,1.0,1.0,2.220446049250313e-16,0.9999999985040425,1.0,0.0,0.9999991678375673,0.9999999996044299,2.2650432640602958e-09,0.3565361262275132,1.0693734786570985e-08,1.0,0.03785033982208286,1.0,4.551914400963142e-15,0.00010701387764577852,0.0001299255943596478,1.0,0.9999999999936849,0.9999999999999996,0.0,1.0,1.0,0.13360685655772264,1.0,0.9996988243808799,1.0,1.0,0.9882591112280137,0.0,0.0,5.573730366137397e-11,1.0,1.0,1.0,0.0,0.9999999981477481,1.0,0.9774589051981063,1.0,0.99999999999997,1.0,1.0,0.9109640549328647,1.4310774787418268e-13,0.0,1.0,1.0,1.0,1.0,1.0,0.0,0.9814915593419258,1.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.9999999735419358,2.0301538228295612e-12,0.9999999993866077,0.5489317590107377,1.0,0.0,0.9999999813806282,0.9999999026760565,1.0,0.0,1.0,0.9094029415721272,0.0,1.0,0.9654027877061505,0.9808195608651251,0.0,0.0,0.9999977456025537,1.0,0.0,0.9204635908914672,0.0,0.0,0.9808349484885635,1.0,1.0,0.9999999992728433,1.0,1.4860163491947631e-06,0.00025836204253826267,1.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,1.0,0.0,1.0,1.0,0.9999999944050656,0.0,0.0,1.0,0.9999999830380772,0.0,0.8557421961874316,0.0,1.0,1.0,1.0,1.0,1.0,1.0,6.131095631189964e-11,0.9999437167794724,1.0,0.20107982109142464,0.0,0.0,1.0,0.005901275780101245,0.9999997620061168,0.014151851345717392,1.0,1.0,0.0,1.0,0.0,5.6454827563889864e-06,1.0,0.0,0.0,1.0,1.0,1.0,0.5688022434759248,0.9999999926546038,1.0,0.0,0.0,1.0,1.0,0.0,1.0,0.02572446344520285,0.9988915320494959,1.0,1.0,1.0,1.0,0.040610571637394455,0.7432038657167568,0.04997034374910292,0.20166883708533945,0.3680700228352971,0.023335418552378973,6.661338147750939e-16,0.9999999999947776,1.0,1.0,0.0,0.0,0.9999999999999634,0.0,0.0,1.0,0.0,0.9999999999974494,1.0,0.0,0.9999996289415393,1.0,0.9945533212786726,1.0,1.0,0.9999999999999999,0.003963722388499957,0.0,1.0,0.9965228110843182,0.0,0.9998849452533929,0.0,1.0,0.9978087193513476,1.0,0.0,1.0,1.0,0.9997520477910156,1.0,0.9999995302440982,1.0,1.0,0.0,1.0,1.0,0.0,0.18860418958728242,1.0,0.9894318915984716,1.0,0.5958552834371103,0.0,0.9991250512546973,1.0,0.9999997057285267,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,0.0,8.278577823261912e-11,1.0,1.4163416185430933e-06,1.0,0.0,0.9999999999999982,1.0,0.0,1.0,0.40048398435606924,1.1102230246251565e-16,1.0,0.9999934081425949,1.0,0.0,1.0,1.0,1.0,1.0,0.9999999999973355,0.0,0.999999999999993,1.0,0.9961456279106113,0.8683265529245581,0.7842827563144906,0.0,0.0,1.0,0.9999999999996311,1.0,1.0,1.0,0.0,1.0,0.9659191211215585,0.9992795840824131,1.0,1.0,1.0,1.0,1.0,

In [60]:
print("degreesOfFreedom: " + str(r.degreesOfFreedom))
 

degreesOfFreedom: [727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 727, 7

In [61]:

print("statistics: " + str(r.statistics))

statistics: [1228.9327910353002,632.679872251771,870.4451037847506,1616.4913047801747,722.8146162598016,657.1887375790625,640.3320730267493,809.840241966198,1171.6379604743167,735.8737657431888,1055.928985544375,1400.9904927547518,1353.2832290284196,560.2952111948423,1145.6052181197147,922.4707140858619,946.439446930545,1473.2673534529615,1237.9467374989936,674.701647611224,1008.4233798167072,682.620820487357,828.3615999819369,772.2111539131702,614.5528659519282,1292.6338774392389,1218.5959622593286,712.7140415778192,660.3259567773354,723.8182299523469,1411.890683317175,882.4465643571069,1234.7444033113934,994.3092798227384,1024.6979084246632,1422.131260311599,940.9625899310046,1327.3183592345772,907.9536001368189,1360.2349039357412,382.1094912583183,851.0629491035078,833.2810475630702,595.3805237172526,768.8364262502836,576.365230894223,1528.9796501205888,1391.327691535154,422.4256048751766,438.86500374018595,687.4722670487827,550.1941698675759,1009.9816231558659,406.4332646910254,1139.7663796513184,266.54389363343506,522.0014892069672,344.37892904584396,925.6491080696725,303.7520526412232,1048.7125934625205,366.96687041056305,1298.3288529894455,419.539726261,738.4116161872462,1313.1404450862908,457.7778278488592,567.9058212689972,490.0427680140708,502.7967802318159,228.12894248010818,299.1936067601684,779.6045235566015,497.68033269176857,470.3891379777135,755.0950488150492,517.8930322410167,294.71057504066954,1548.994240368423,463.4640339172884,780.5356569753521,334.625405794014,447.68668693538797,242.67256066072707,1648.1895814032769,1604.3388745413495,348.29595683399276,262.4246758765041,658.4554762967551,522.2999492891836,739.461571905943,215.09354656331982,345.2483632785601,405.438352100516,261.426438286465,981.7055891304129,209.47922979788243,646.5900009062635,1410.3264744188284,328.51690566429556,833.1963749702361,291.9891890006327,617.8964785022547,244.80330224965715,839.1484493139544,418.57546516747874,587.5171720760165,474.8828440378024,230.78176022414655,1089.3478947021786,203.25321645899552,781.168424890255,544.987559858078,381.75226787037235,266.77573098437625,1756.3692808482233,627.0505943474799,708.0930172710705,236.29970146938237,248.8582760041177,257.45674931684573,1081.5029939991477,523.0810841040454,264.74849038256906,1247.2630539693469,558.7321963559209,516.5563608327361,973.283398018148,740.4383906016071,961.1700445334238,304.05026441225493,796.1451815659606,411.550735424457,1063.0061090077415,876.6940207628786,874.5589328833246,384.9186981045496,497.8439414346997,461.56504658726146,1151.1951835180723,175.34375489200272,207.76626661243503,769.4421721627087,312.4852284745925,603.3164478569529,305.45568060247615,258.9261180548342,643.3799168423369,1805.8886282999251,1251.5834115075934,1000.6540030381811,412.16603331037703,323.24608059300203,288.4495864667745,1545.513104985572,524.155371422649,182.15289808922327,652.6164669227711,427.37425545599507,476.46396687156226,328.71846044080723,425.09233248950886,676.2106541192139,1041.1780864586854,1206.1342815310568,334.36184498712595,320.8931922891194,299.45120416171466,347.9809678104486,201.98654523720072,1711.16625364567,649.7227971308416,356.33695504124404,196.43257063602212,1345.5423914786875,182.60807346133942,1239.670682688504,1177.1077311324614,202.2031959577821,1602.7246962519143,2319.053810264041,168.6265261230894,2188.2036028761904,1679.930919788245,136.76308829773603,395.33098957177185,538.2118957971315,1023.6232120906661,518.6776977374014,721.657283055492,393.3980685167931,1181.3686734664086,536.276580660087,545.6191820379534,147.38916545927017,1557.3465473524964,256.27744118793373,676.5605178284804,1198.6899454929485,216.17396158077977,659.2636707096158,650.2393290842185,1823.4681344501128,1464.9294646509302,565.2721810028695,180.19569999375472,1278.6684438482564,673.9794189599077,2131.109004924022,1736.9348538821762,650.2276717958198,389.96996215697123,133.54263131941212,519.5083494644994,338.58886731997967,919.2593689036893,866.8068801905293,212.408